In [192]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, Label, LabelSet, Range1d, Text, Whisker
from bokeh.transform import factor_cmap
from scipy.stats import linregress
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
from bokeh.palettes import Category20

file_path= "/Users/ajaychandraas/Desktop/Datasets written assignment-231-4 2/"
train_path= file_path + "train.csv"; train_read = pd.read_csv(train_path); train_df= pd.DataFrame(train_read)
ideal_path= file_path + "ideal.csv"; ideal_read= pd.read_csv(ideal_path); ideal_df= pd.DataFrame(ideal_read)
test_path= file_path + "test.csv"; test_read= pd.read_csv(test_path); test_df= pd.DataFrame(test_read)

# Train and test descriptive statistics
def train_test_desc():
    tr_desc= train_read.describe(); print(tr_desc)
    tst_desc= test_read.describe(); print(tst_desc)

# Train outlier values
def train_outliers():
    outier_dict= {}
    col_name= train_df.columns.tolist()
    for rec in col_name:
        bounds_dict= {}
        if(rec!= 'x' and rec!= 'index'):
            y_values= train_df.loc[:,rec].values.tolist()
            # Finding IQR
            Q1 = np.percentile(y_values, 25, interpolation = 'midpoint')
            Q3 = np.percentile(y_values, 75, interpolation = 'midpoint')
            IQR = Q3 - Q1
            # Finding upper and lower bounds
            uppr_bound= Q3+1.5*IQR
            lowr_bound= Q1-1.5*IQR
            upprcount= 0
            lwrcount= 0
            # Counting the outliers in each column
            for y_rec in y_values:
                if(y_rec> uppr_bound):
                    upprcount+= 1
                elif(y_rec< lowr_bound):
                    lwrcount+= 1
                else:
                    pass     
            bounds_dict['upper bound']= upprcount     
            bounds_dict['lower bound']= lwrcount
            outier_dict[rec]= bounds_dict
    print(outier_dict)  

    
# Test outlier values
def test_outliers():
    outier_dict= {}
    bounds_dict= {}
    y_values= test_df.loc[:,'y'].values.tolist()
    # Finding IQR
    Q1 = np.percentile(y_values, 25, interpolation = 'midpoint')
    Q3 = np.percentile(y_values, 75, interpolation = 'midpoint')
    IQR = Q3 - Q1
    # Finding upper and lower bounds
    uppr_bound= Q3+1.5*IQR
    lowr_bound= Q1-1.5*IQR
    upprcount= 0
    lwrcount= 0
    for y_rec in y_values:

        if(y_rec> uppr_bound):
            upprcount+= 1
        elif(y_rec< lowr_bound):
            lwrcount+= 1
        else:
            pass 
        bounds_dict['upper bound']= upprcount     
        bounds_dict['lower bound']= lwrcount
        outier_dict['y']= bounds_dict
    print(outier_dict)
   
# Visualizing Train box plots
def train_box_plots():
    train_list= train_df.columns.tolist()
    const_dict= {}; const_name= []; const_y= []; data_dict= {}
    q1_list= []; q2_list= []; q3_list= []
    for rec1 in train_list:
        if(rec1!= 'x' and rec1!= 'index'):
            name_lst= [rec1]*len(train_df)
            y_lst= train_df[rec1].tolist()
            const_name.extend(name_lst)
            const_y.extend(y_lst)
    const_dict['Train']= const_name
    const_dict['y']= const_y
    df= pd.DataFrame(const_dict)

    trains = df.Train.unique()
    qs = df.groupby("Train").y.quantile([0.25, 0.5, 0.75])
    qs = qs.unstack().reset_index()
    qs.columns = ["Train", "q1", "q2", "q3"]
    df = pd.merge(df, qs, on="Train", how="left")
    # compute IQR outlier bounds
    iqr = df.q3 - df.q1
    df["upper"] = df.q3 + 1.5*iqr
    df["lower"] = df.q1 - 1.5*iqr

    source = ColumnDataSource(df)
    p = figure(x_range=trains, tools="", toolbar_location=None,
               title="Train Box plots",
               background_fill_color="#eaefef", y_axis_label="train")
    # outlier range
    whisker = Whisker(base="Train", upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)
    # quantile boxes
    color1 = ["pink", "yellow", "lightgreen","orange", "red"]  
    cmap = factor_cmap("Train", color1, trains)
    p.vbar("Train", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
    p.vbar("Train", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")
    # outliers
    outliers = df[~df.y.between(df.lower, df.upper)]
    p.scatter("Train", "y", source=outliers, size=6, color="black", alpha=0.3)
    p.xgrid.grid_line_color = None
    p.axis.major_label_text_font_size="18px"
    p.axis.axis_label_text_font_size="18px"
    try:
        show(p)
    except Exception as _error:
        print(f"Error: Unable to generate the graph: {_error}") 
train_box_plots()

# Visualizing Test box plot
def test_box_plot():
    const_dict= {}
    name_lst= ['y']*len(test_df)
    y_lst= test_df['y'].tolist()
    const_dict['Test']= name_lst
    const_dict['y']= y_lst
    df= pd.DataFrame(const_dict)
    tests = df.Test.unique()
    qs = df.groupby("Test").y.quantile([0.25, 0.5, 0.75])
    qs = qs.unstack().reset_index()

    qs.columns = ["Test", "q1", "q2", "q3"]
    df = pd.merge(df, qs, on="Test", how="left")
    # compute IQR outlier bounds
    iqr = df.q3 - df.q1
    df["upper"] = df.q3 + 1.5*iqr
    df["lower"] = df.q1 - 1.5*iqr

    source = ColumnDataSource(df)
    p = figure(x_range=tests, tools="", toolbar_location=None,
                title="Test Box plot",
                background_fill_color="#eaefef", y_axis_label="test")
    p.title.text_font_size = '15pt'
    # outlier range
    whisker = Whisker(base="Test", upper="upper", lower="lower", source=source)
    whisker.upper_head.size = whisker.lower_head.size = 20
    p.add_layout(whisker)
    # quantile boxes
    color1 = ["pink", "yellow", "lightgreen","orange", "red"]  
    cmap = factor_cmap("Test", color1, tests)
    p.vbar("Test", 0.7, "q2", "q3", source=source, color=cmap, line_color="black")
    p.vbar("Test", 0.7, "q1", "q2", source=source, color=cmap, line_color="black")
    # outliers
    outliers = df[~df.y.between(df.lower, df.upper)]
    p.scatter("Test", "y", source=outliers, size=6, color="black", alpha=0.3)
    p.xgrid.grid_line_color = None
    p.axis.major_label_text_font_size="18px"
    p.axis.axis_label_text_font_size="18px"
    show(p) 
            
# Checking for duplicates in train
def find_dup_train():
    dup_dict= {}
    col_name= train_df.columns.tolist()  
    for rec in col_name:
        total_items= len(train_df[rec])
        uniq_items= len(train_df[rec].unique())
        dup_items= total_items- uniq_items
        dup_dict[rec]= dup_items 
    print(dup_dict)

# Checking for duplicates in test  
def find_dup_test():
    dup_dict= {}
    col_name= test_df.columns.tolist()
    
    for rec in col_name:
        total_items= len(test_df[rec])
        uniq_items= len(test_df[rec].unique())
        dup_items= total_items- uniq_items
        dup_dict[rec]= dup_items
    print(dup_dict)

# Checking for missing values in train and test
def missing_values():    
    train_num_missing = train_read.isnull().sum()
    print(train_num_missing)
    test_num_missing = test_read.isnull().sum()
    print(test_num_missing)

# Best fit line for train, ideal and combination [all approachs]
def bestfit_scatter_plots():
    method_dict= {"Regression":['y2','y38','y43','y46'], "Correlation":['y9','y38','y45','y46'], "Least-Squares":['y9','y38','y43','y46']} 
    ideal_list=[]
    train_list= ['y1','y2','y3','y4']
    for key, value in method_dict.items():
        if(key== "Regression"):
            ideal_list= value  
        elif(key== "Correlation"):
            ideal_list= value  
        else:
            ideal_list= value 

        x_values_train= train_df.loc[:,'x'].values.tolist()        
        for train_rec, ideal_rec in zip(train_list, ideal_list): 
            y_values= train_df.loc[:,train_rec].values.tolist()
            p= figure(title= f"{key} Approach: Fitting {train_rec} and {ideal_rec} regression lines")
            p.title.text_font_size = '15pt'
            p.scatter(x_values_train, y_values)
            train_x= train_df.loc[:,'x'].values
            train_y= train_df.loc[:,train_rec].values
            slope1, intercept1, r_value1, p_value1, std_err1= linregress(train_x, train_y)
            xs1 = np.linspace(np.min(train_x), np.max(train_x))
            ys1 = slope1 * xs1 + intercept1
            p.line(xs1, ys1, color='green', line_width=2.5, legend_label='Train regression line')

            ideal_x= ideal_df.loc[:,'x'].values
            ideal_y= ideal_df.loc[:,ideal_rec].values
            slope2, intercept2, r_value2, p_value2, std_err2= linregress(ideal_x, ideal_y)
            xs2 = np.linspace(np.min(ideal_x), np.max(ideal_x))
            ys2 = slope2 * xs2 + intercept2
            p.line(xs2, ys2, color='red', line_width=2.5, legend_label='Ideal regression line')
            show(p) 
        

            
# Bar chart to show ideal and y values [Regression approach] executed for individual train function
def bar_chart_regression():
    main_dict= {}
    train_x_values= train_read.loc[:,'x'].values;train_x_df= pd.DataFrame(train_x_values)
    train_col_name= train_read.columns.tolist()
    for train_rec in train_col_name:
        if(train_rec!= 'x' and train_rec!= 'index'):
            secondary_dict= {}
            train_column= train_read.loc[:,train_rec].values
            # Finding regression line of ideal functions
            slope1, intercept1, r_value1, p_value1, std_err1= linregress(train_x_values, train_column)
            train_x_arr_values= np.array(train_x_values)
            train_y_arr_values= np.array(train_column)
            train_est_y= (slope1 * train_x_arr_values) + intercept1
            train_est_y_df= pd.DataFrame(train_est_y)
            
            ideal_x_values= ideal_read.loc[:,'x'].values;ideal_x_df= pd.DataFrame(ideal_x_values)
            ideal_col_name= ideal_read.columns.tolist()
            for ideal_rec in ideal_col_name:
                if(ideal_rec!= 'x' and ideal_rec!= 'index'):
                    ideal_column= ideal_read.loc[:,ideal_rec].values;
                    # Finding regression line of ideal functions
                    slope2, intercept2, r_value2, p_value2, std_err2= linregress(ideal_x_values, ideal_column)
                    ideal_x_arr_values= np.array(ideal_x_values)
                    ideal_y_arr_values= np.array(ideal_column)
                    ideal_est_y= (slope2 * ideal_x_arr_values) + intercept2
                    ideal_est_y_df= pd.DataFrame(ideal_est_y)
                    # Train and ideal regr diff
                    diff= sum(abs(train_est_y- ideal_est_y))
                    secondary_dict[ideal_rec]= diff
            min_value= min(secondary_dict.values())
            min_keys = [key for key in secondary_dict if secondary_dict[key]==min_value]
            
            key_list= list(secondary_dict.keys())
            value_list = np.log10(list(secondary_dict.values()))
            title= f"Sum of absolute deviation of regression values for '{train_rec}' train with every ideal functions"
            p = figure(title= title, x_range=key_list, plot_width=1300, plot_height=300)
            p.yaxis.major_label_text_font_size = '8pt'; p.xaxis.major_label_text_font_size = '11pt'
            p.title.text_font_size = '13pt'
            
            index = key_list.index(min_keys[0])
            colors = ['blue' if i != index else 'red' for i in range(len(value_list))]
            p.vbar(x= key_list, top= value_list, width=0.5, color=colors)
            label = Label(x=1, y=10, text='Min Value', text_font_size='13pt', text_color='red')
            p.add_layout(label)
            label.x = 45; label.y = 6; label.y_offset = 1
            show(p)
      

           
# Bar chart to show ideal and y values [Correlation approach] executed for individual train function
def bar_chart_correlation():
    ideal_dict= {'y1': 'y9', 'y2': 'y38', 'y3': 'y45', 'y4': 'y46'}
    train_col_name= train_read.columns.tolist()
    for train_rec in train_col_name:     
        if(train_rec!= 'x' and train_rec!= 'index'):
            secondary_dict= {}
            train_column= train_read.loc[:,train_rec].values; 
            train_df= pd.DataFrame(train_column, columns = ['y'])
            ideal_col_name= ideal_read.columns.tolist()
            for ideal_rec in ideal_col_name:
                if(ideal_rec!= 'x' and ideal_rec!= 'index'):
                    ideal_column= ideal_read.loc[:,ideal_rec].values
                    ideal_df= pd.DataFrame(ideal_column, columns = ['y'])
                    calc_correl= train_df['y'].corr(ideal_df['y'])
                    secondary_dict[ideal_rec]=calc_correl
            max_value= max(secondary_dict.values())
            max_keys = [key for key in secondary_dict if secondary_dict[key]==max_value]
            key_list= list(secondary_dict.keys())
            
            value_list1= list(secondary_dict.values())
            value_list2= [i * 100000 for i in value_list1]
            replace_neg = [max(0, i) for i in value_list2]
            place_zero = [i if i != 0 else 1 for i in replace_neg]
            new_value_list = np.log10(place_zero).tolist()
                                  
            title= f"Correlation values of '{train_rec}' train with every ideal functions"
            p = figure(title= title, x_range=key_list, plot_width=1300, plot_height=300)
            p.yaxis.major_label_text_font_size = '8pt'; p.xaxis.major_label_text_font_size = '11pt'
            p.title.text_font_size = '13pt'
            index = key_list.index(max_keys[0])
            colors = ['blue' if i != ideal_dict[train_rec] else 'red' for i in key_list]
            
            p.vbar(x= key_list, top= new_value_list, width=0.5, color=colors)
            label = Label(x=1, y=10, text='Max Value', text_font_size='10pt', text_color='red')
            p.add_layout(label)
            label.x = 47; label.y = 4.75; label.y_offset = 1
            show(p)
            
 

# Bar chart to show ideal and y values [Least-squares approach] executed for individual train function
def bar_chart_leastSquares():
    main_dict= {}
    train_x_values= train_read.loc[:,'x'].values
    train_col_name= train_read.columns.tolist()
    for train_rec in train_col_name:     
        if(train_rec!= 'x' and train_rec!= 'index'):
            secondary_dict= {}
            train_column= train_read.loc[:,train_rec].values    
            ideal_x_values= ideal_read.loc[:,'x'].values;ideal_x_df= pd.DataFrame(ideal_x_values)
            ideal_col_name= ideal_read.columns.tolist()
            for ideal_rec in ideal_col_name:
                if(ideal_rec!= 'x' and ideal_rec!= 'index'):
                    ideal_column= ideal_read.loc[:,ideal_rec].values;
                    # Train and ideal diff
                    diff= sum((train_column- ideal_column)**2)
                    secondary_dict[ideal_rec]= diff

            min_value= min(secondary_dict.values())
            min_keys = [key for key in secondary_dict if secondary_dict[key]==min_value]

            key_list= list(secondary_dict.keys())
            value_list = np.log10(list(secondary_dict.values()))
        
            title= f"Sum of Least square values of '{train_rec}' train with every ideal functions"
            p = figure(title= title, x_range=key_list, plot_width=1300, plot_height=300)
            p.yaxis.major_label_text_font_size = '8pt'; p.xaxis.major_label_text_font_size = '11pt'
            p.title.text_font_size = '13pt'

            index = key_list.index(min_keys[0])
            colors = ['blue' if i != index else 'red' for i in range(len(value_list))]
            p.vbar(x= key_list, top= value_list, width=0.5, color=colors)
            label = Label(x=1, y=10, text='Min Value', text_font_size='13pt', text_color='red')
            p.add_layout(label)
            show(p)

# Pie chart that shows the results of all the approaches by comparing test-ideal mapping 
def counter_pie_chart(): 
    reg_fields= ["y2 (2.099)", "y38 (0.707)", "y43 (0.706)", "y46 (0.706)", "N/A"]  
    reg_percentages= [36.6, 8.2, 18.3, 8.2, 28.7]  
    reg_title= "Regression approach"

    corr_fields= ["y9 (0.699)", "y38 (0.707)", "y45 (14.848)", "y46 (0.706)", "N/A"]
    corr_percentages= [10, 7.5, 60.8, 7.5, 14.2]
    corr_title= "Correlation approach"

    ls_fields= ["y9 (0.699)", "y38 (0.707)", "y43 (0.706)", "y46 (0.706)", "N/A"]
    ls_percentages= [12.2, 9.1, 20.4, 9.1, 49.2]
    ls_title= "Least-squares approach"

    method_dict= ["regr", "corr", "ls"]
    field_var= [];pct_var= []; title_var="";
    for rec in method_dict:
        if(rec== "regr"):
            field_var= reg_fields; pct_var= reg_percentages; title_var= reg_title
        elif(rec== "corr"):
            field_var= corr_fields; pct_var= corr_percentages; title_var= corr_title
        else:
            field_var= ls_fields; pct_var= ls_percentages; title_var= ls_title

        # then, we will instantiate the figure object   
        graph1 = figure(title = f"{title_var}: Test-ideal mapping count") 
        graph1.title.text_font_size = "18pt"
        # name of the fields for investment  
        fields = field_var
        # % tage weightage of the sectors  
        percentages = pct_var
        # formula for converting percentage into radians  
        radians1 = [math.radians((percent / 100) * 360) for percent in percentages]   
        # Generating the start angle values  
        start_angle = [math.radians(0)]  ; prev = start_angle[0]  
        for k in radians1[:-1]:  
            start_angle.append(k + prev)  
            prev = k + prev    
        # generating the end angle values  
        end_angle = start_angle[1:] + [math.radians(0)]    
        # initiate the center of the pie chart  
        x = 0 ;y = 0    
        # then, we will initiate the radius of the glyphs  
        radius = 1    
        # now, generate the color of the wedges  
        color1 = ["pink", "yellow", "lightgreen","orange", "red"]    
        # now, we will plot the graph  
        for k in range(len(fields)):  
            graph1.wedge(x, y, radius,  
                        start_angle = start_angle[k],  
                        end_angle = end_angle[k],  
                        color = color1[k],  
                        legend_label = fields[k]) 
        graph1.legend.label_text_font_size = '19pt'
        # At last, we will display the graph  
        show(graph1) 




 





            
            
            
            
            
            
    

